In [2]:
#=========================================================================
# load up the libraries
#=========================================================================
import pandas  as pd
import numpy   as np
import xgboost as xgb

#=========================================================================
# read in the data
#=========================================================================
import pandas as pd
import zipfile

# Set the path to your ZIP file
zip_path = r'C:/Users/alial/OneDrive/Desktop/LHL/applications of ml assignments/predict-student-performance-from-game-play.zip'

# Open and inspect the ZIP file
with zipfile.ZipFile(zip_path, 'r') as z:
    print(z.namelist())  # Lists files inside the ZIP to confirm the names

    # Read train.csv from the ZIP
    with z.open('train.csv') as f_train:
        train_data = pd.read_csv(f_train, index_col=0)

    # Read test.csv from the ZIP
    with z.open('test.csv') as f_test:
        test_data = pd.read_csv(f_test, index_col=0)

['jo_wilder/__init__.py', 'jo_wilder/competition.cpython-37m-x86_64-linux-gnu.so', 'jo_wilder_310/__init__.py', 'jo_wilder_310/competition.cpython-310-x86_64-linux-gnu.so', 'sample_submission.csv', 'test.csv', 'train.csv', 'train_labels.csv']


In [9]:
import pandas as pd


train_csv = r'C:/Users/alial/OneDrive\Desktop/LHL/applications of ml assignments/train.csv'
test_csv  = r'C:/Users/alial/OneDrive\Desktop/LHL/applications of ml assignments/test.csv'

# Load CSVs directly
df_train = pd.read_csv(train_csv)
df_test  = pd.read_csv(test_csv)

# Preview to confirm
print(df_train.shape)
print(df_train.head())

(26296946, 20)
          session_id  index  elapsed_time      event_name   name  level  page  \
0  20090312431273200      0             0  cutscene_click  basic      0   NaN   
1  20090312431273200      1          1323    person_click  basic      0   NaN   
2  20090312431273200      2           831    person_click  basic      0   NaN   
3  20090312431273200      3          1147    person_click  basic      0   NaN   
4  20090312431273200      4          1863    person_click  basic      0   NaN   

   room_coor_x  room_coor_y  screen_coor_x  screen_coor_y  hover_duration  \
0  -413.991405  -159.314686          380.0          494.0             NaN   
1  -413.991405  -159.314686          380.0          494.0             NaN   
2  -413.991405  -159.314686          380.0          494.0             NaN   
3  -413.991405  -159.314686          380.0          494.0             NaN   
4  -412.991405  -159.314686          381.0          494.0             NaN   

                            text   

In [10]:
print(df_train.head())
print(df_train.columns)
print(df_train.info())
print(df_train.isnull().sum())

          session_id  index  elapsed_time      event_name   name  level  page  \
0  20090312431273200      0             0  cutscene_click  basic      0   NaN   
1  20090312431273200      1          1323    person_click  basic      0   NaN   
2  20090312431273200      2           831    person_click  basic      0   NaN   
3  20090312431273200      3          1147    person_click  basic      0   NaN   
4  20090312431273200      4          1863    person_click  basic      0   NaN   

   room_coor_x  room_coor_y  screen_coor_x  screen_coor_y  hover_duration  \
0  -413.991405  -159.314686          380.0          494.0             NaN   
1  -413.991405  -159.314686          380.0          494.0             NaN   
2  -413.991405  -159.314686          380.0          494.0             NaN   
3  -413.991405  -159.314686          380.0          494.0             NaN   
4  -412.991405  -159.314686          381.0          494.0             NaN   

                            text    fqid          

In [37]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Step 1: Load datasets
train_csv = 'C:/Users/alial/OneDrive/Desktop/LHL/applications of ml assignments/train.csv'
test_csv  = r'C:/Users/alial/OneDrive/Desktop/LHL/applications of ml assignments/test.csv'
labels_csv = r'C:/Users/alial/OneDrive/Desktop/LHL/applications of ml assignments/train_labels.csv'

df_train = pd.read_csv(train_csv)
df_test = pd.read_csv(test_csv)
df_labels = pd.read_csv(labels_csv)

# Step 2: Clean 'session_id' in df_labels by removing suffixes like '_q<number>'
df_labels['session_id'] = df_labels['session_id'].str.replace(r'_q\d+$', '', regex=True)

# Step 3: Ensure 'session_id' columns are same type (string) in both dataframes
df_train['session_id'] = df_train['session_id'].astype(str)
df_labels['session_id'] = df_labels['session_id'].astype(str)

In [38]:
# Step 4: Find common session_ids between train and labels to avoid mismatches
common_sessions = set(df_train['session_id']).intersection(set(df_labels['session_id']))
print(f"Number of common session_ids after cleaning: {len(common_sessions)}")

Number of common session_ids after cleaning: 23562


In [46]:
sample_size = 2000  # or smaller like 1000
sample_sids = df_labels_reduced['session_id'].sample(sample_size, random_state=42).unique()

df_train_sample = df_train[df_train['session_id'].isin(sample_sids)]
df_labels_sample = df_labels_reduced[df_labels_reduced['session_id'].isin(sample_sids)]

In [47]:
# Example relevant columns from training data (adjust as needed)
relevant_train_cols = ['session_id', 'elapsed_time', 'event_name', 'level', 'room_coor_x', 'room_coor_y']

# From labels, probably just 'session_id' and the target 'correct'
relevant_label_cols = ['session_id', 'correct']

df_train_sample = df_train[df_train['session_id'].isin(sample_sids)][relevant_train_cols]
df_labels_sample = df_labels_reduced[df_labels_reduced['session_id'].isin(sample_sids)][relevant_label_cols]

In [48]:
df_train_merged = df_train_sample.merge(df_labels_sample, on='session_id', how='inner')
print(f"Merged data shape: {df_train_merged.shape}")

Merged data shape: (38419182, 7)


In [50]:
df_train_merged['event_name'] = df_train_merged['event_name'].astype('category')
df_train_merged['level'] = df_train_merged['level'].astype('int8')
df_train_merged['correct'] = df_train_merged['correct'].astype('int8')

In [51]:
# Drop columns that won't be useful or cause issues (adjust as needed)
cols_to_drop = ['session_id', 'text', 'page']  # example non-numeric or unstructured columns
df = df_train_merged.drop(columns=[col for col in cols_to_drop if col in df_train_merged.columns])

# Handle missing values for numeric columns (example: fill with 0)
for col in df.select_dtypes(include=['float', 'int']).columns:
    df[col] = df[col].fillna(0)

# Convert categorical columns to numeric codes (if any)
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
for col in categorical_cols:
    df[col] = df[col].astype('category').cat.codes

# Define features (X) and target (y)
X = df.drop(columns=['correct'])
y = df['correct']

In [ ]:
from sklearn.model_selection import train_test_split
from scipy.sparse import csr_matrix

print(type(X))  # Is it sparse?

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=None

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Initialize model
model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)

# Train model
model.fit(X_train, y_train)

# Predict on validation set
y_pred = model.predict(X_val)

# Evaluate
print("Validation Accuracy:", accuracy_score(y_val, y_pred))
print("Classification Report:\n", classification_report(y_val, y_pred))

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

feature_importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(10,6))
feature_importances.head(20).plot(kind='bar')
plt.title('Top 20 Feature Importances')
plt.show()

In [ ]:
df_X_train = pd.DataFrame(X_train)
df_X_train.to_csv("X_train.csv", index=False)

# Save y_train (if it's 1D, just convert directly)
df_y_train = pd.DataFrame(y_train, columns=["target"])
df_y_train.to_csv("y_train.csv", index=False)

# Similarly for validation sets:
pd.DataFrame(X_val).to_csv("X_val.csv", index=False)
pd.DataFrame(y_val, columns=["target"]).to_csv("y_val.csv", index=False)